In [1]:
import cv2
import numpy as np

# Initial configurations
VIDEO_PATH = 'media/Figures moving.mp4'
TEMPLATE_PATH = 'media/template_figure.jpg'
SCALE_FACTOR = 0.5  

# Template loading
template = cv2.imread(TEMPLATE_PATH, cv2.IMREAD_GRAYSCALE)
template = cv2.resize(template, (0, 0), fx=SCALE_FACTOR, fy=SCALE_FACTOR)
template_height, template_width = template.shape

# ORB configurations
orb = cv2.ORB_create()
keypoints_template, descriptors_template = orb.detectAndCompute(template, None)

# Video Capture
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)

try:
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame = cv2.resize(frame, (0, 0), fx=SCALE_FACTOR, fy=SCALE_FACTOR)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detection of KeyPoints for the Frame.
        keypoints_frame, descriptors_frame = orb.detectAndCompute(gray_frame, None)

        if descriptors_frame is not None and descriptors_template is not None:
            
            # BF Matcher and matching.
            bf = cv2.BFMatcher()

            # UNCOMMENT FOR BETTER RESULTS:
            # Normalization configuration depending on the type of data from the template.
            # norm_type = cv2.NORM_HAMMING if descriptors_template.dtype == np.uint8 else cv2.NORM_L2
            # bf = cv2.BFMatcher(norm_type, crossCheck=True)

            matches = bf.match(descriptors_template, descriptors_frame)

            # Anonymous function to do a quick filtering according to distance. 
            # "x" can be renamed, "match" as an example.
            matches = sorted(matches, key=lambda x: x.distance)

            # Drawing max. 10 matches.
            frame = cv2.drawMatches(template, 
                                    keypoints_template, 
                                    frame, 
                                    keypoints_frame, 
                                    matches[:10], 
                                    None, 
                                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Template showing with a resized size.
        template_resized = cv2.resize(cv2.cvtColor(template, cv2.COLOR_GRAY2BGR), 
                                      (int(template_width * SCALE_FACTOR), 
                                      int(template_height * SCALE_FACTOR)))

        cv2.imshow('Feature Matching', frame)

        # The following can be cv2.waitKey(0) so that it moves with a pressed key.
        if cv2.waitKey(int(1000 // fps)) == ord('q'):
            break

except Exception as error:
    print(error)

finally:
    cap.release()
    cv2.destroyAllWindows()
